## AI Foundry Onboarding 


### Create your AI Foundry Project 
1. Go to [ai.azure.com](https://ai.azure.com/)
2. Create a new Azure AI Project


![Creating an AI Project](./images/aifoundry01.png)
Click on "Create"...

![Creating an AI Project02](./images/aifoundry02.png)
Choose "AI Foundry Resource"...

![Creating an AI Project03](./images/aifoundry03.png)
You can use any region to create your AI Foundry Project in. \
Make sure the models you would like to use are available in the region of your choice. \
You can use the model availability table named "Global standard model availability", in [*"Azure OpenAI in Azure AI Foundry Models" documentation*"](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models?tabs=global-standard%2Cstandard-chat-completions) for checking which models are available in which region. This is continously updated. For the purposes of hackathon sweden-central would have all models we would need to use. 


![Creating an AI Project04](./images/aifoundry04.png)

Once you click on "create" your projects starts to be provisioned. It may take 1-2 mins for your project to be ready.

![Creating an AI Project05](./images/aifoundry05.png)

Once created you will land on your project page with a project key. 
AI Foundry project key is used when you are using the Foundry SDK which facilitates services integration to AI Foundry such as AI Search. We will not be using Foundry SDK in the workshop. 


![Creating an AI Project06](./images/aifoundry06.png)

Next, we will deploy LLM's on AI Foundry, we will use in our development. \
Choose model catalog and search for gpt-4.1 which is the latest model from openai being offered on Azure OpenAI service replacing gpt-4o. 



![Creating an AI Project07](./images/aifoundry07.png)

Deploy the model...We will not need to change default parameters e.g. model version, quoata or filter so keep them as they are given by default.

![Creating an AI Project08](./images/aifoundry08.png)

Copy your API key for later use...

![Creating an AI Project09](./images/aifoundry09.png)

When the model is deployed you will be given code samples as to how to make an API call for LLM generations. Copy your API endpoint from the sample code. 

![Creating an AI Project10](./images/aifoundry10.png)

Repeat the same deployment task for o4-mini, which is a reasoning model we will later use for more sophisticated tasks where "reasoning" will be required e.g. checking a received customer contract clause for compliance or for planning e.g. decomposing a complex task into smaller tasks and executing them with another "workhorse" model such as gpt-4.1. \
While deploying these models, please keep deployment names simple. e.g. if you are deploying a gpt-4.1 keep the deployment name as "gpt-4.1", or if you are deploying o4-mini, keep your deployment name as "o4-mini" and the embedding models deployment name as "text-embedding-3-small".

![Creating an AI Project11](./images/aifoundry11_embedv3.png)

Repeat the same model deployment tasks for text-embedding-3-small

![Creating an AI Project12](./images/aifoundry12_final_status.png)

Once gpt-4.1, o3-mini and embedv3-small are deployed we are good to go. 
Comfirm successful deployments on Azure AI Foundry "Models & Assets" under my-assets...


**References**:
- [Create a Project under Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/create-projects?tabs=ai-foundry&pivots=fdp-project)
- [Quickstart - Get started with Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-foundry/quickstarts/get-started-code?tabs=azure-ai-foundry&pivots=fdp-project)
- [Microsoft Learn - Azure AI Foundry Training](https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/)



### Confirm Operation 

Next we will send simple API calls to confirm successful operation...

#### Testing gpt-4.1 deployment

In [ ]:
#!/usr/bin/env python3
import os
from openai import AzureOpenAI

# ── 1. env + client ───────────────────────────────────────────────────────

client = AzureOpenAI(
    api_key        = "FXT...", #Type your API_key here. If you didn't record it earlier go to "Models and Endpoints", choose the deployment e.g. gpt-4.1 and copy your api key and paste it here...
    azure_endpoint = "https://xxx.cognitiveservices.azure.com", #Type your API_endpoint here. If you didn't record it earlier go to your deployment again and copy the endpoint from the code sample, not the target URI...
    api_version    = "2024-12-01-preview",  # e.g. 2024-06-01-preview
)
DEPLOYMENT = "gpt-4.1"       # your model deployment name

# ── 2. system-and-user messages ───────────────────────────────────────────
messages = [
    {"role": "system", "content": "You are a concise astrophysics tutor who explains concepts in plain English."},
    {"role": "user",   "content": "What is a black hole?"}
]

# ── 3. one-shot call ──────────────────────────────────────────────────────
response = client.chat.completions.create(
    model       = DEPLOYMENT,
    messages    = messages,
    temperature = 0.7,
    max_tokens  = 256,
)

print(response.choices[0].message.content.strip())



A black hole is a region in space where gravity is so strong that nothing—not even light—can escape from it. This happens when a massive star collapses under its own gravity at the end of its life. The boundary around a black hole is called the "event horizon." Once something crosses this boundary, it cannot get out. Black holes are invisible, but we can detect them by how they affect nearby stars and gas.


#### Testing o4-mini deployment

In [ ]:
#!/usr/bin/env python3
import os
from openai import AzureOpenAI

# ── 1. Load env & create client ───────────────────────────────────────────
         
client = AzureOpenAI(
    api_key        = "FXT...", #Type your API_key here. If you didn't record it earlier go to "Models and Endpoints", choose the deployment e.g. gpt-4.1 and copy your api key and paste it here...
    azure_endpoint = "https://xxx.cognitiveservices.azure.com", #Type your API_endpoint here. If you didn't record it earlier go to your deployment again and copy the endpoint from the code sample, not the target URI...
    # reasoning models live on a preview API version (adjust if needed)
    api_version    = "2025-04-01-preview",
)

DEPLOYMENT = "o4-mini"

# ── 2. Build the input messages ───────────────────────────────────────────
messages = [
    {"role": "developer", "content": "You are a concise physics tutor."},
    {"role": "user",      "content": "What is a black hole?"}
]

# ── 3. Single Responses-API call ──────────────────────────────────────────
response = client.responses.create(
    model               = DEPLOYMENT,
    input               = messages,            # list of role/content dicts
    reasoning           = {"effort": "medium"},# low | medium | high
    max_output_tokens   = 1024                 # covers reasoning + visible answer
)

print(response.output_text.strip())


A black hole is a region of space where gravity is so intense that nothing—not even light—can escape its pull. Key points:

1. Event horizon: the “point of no return” marking the boundary beyond which escape is impossible.  
2. Singularity: (in classical GR) a point at the center where density and curvature become infinite.  
3. Schwarzschild radius (Rs): for a mass M, Rs = 2GM/c². Anything compressed within this radius becomes a (non-rotating) black hole.  
4. Formation: typically from the gravitational collapse of a massive star after it exhausts its nuclear fuel.  
5. Types:
   • Stellar black holes (5–20 M⊙)  
   • Supermassive black holes (10⁶–10¹⁰ M⊙) at galactic centers  
   • Intermediate and primordial (hypothetical)  
6. Observational signatures: accretion disks (X-ray emission), gravitational lensing, gravitational waves from mergers.  

In General Relativity, black holes are exact solutions (e.g. Schwarzschild, Kerr for rotating cases) characterized entirely by mass, spin a

Notice the way we make a call to the API for the reasoning model o4-mini is different from gpt-4.1. First of all we are using a different API - the responses API whereas we used the completions API previously and secondly reasoning models are different in the way they operate e.g. we define a reasoning effort parameter low | medium | high.

#### Testing embedv3-small deployment


In [9]:
# ── 1. embed the question ─────────────────────────────────────────

from openai import AzureOpenAI
AOAI_VER     = "2024-12-01-preview"
EMBED_DEPLOY = "text-embedding-3-small"
QUESTION = "How are you?"

aoai = AzureOpenAI(
    api_key="FXT...",
    api_version=AOAI_VER,
    base_url=f"https://XXX.cognitiveservices.azure.com/openai/deployments/{EMBED_DEPLOY}"
)


embedding = aoai.embeddings.create(model=EMBED_DEPLOY, input=[QUESTION]).data[0].embedding
print(len(embedding),embedding)


1536 [0.027567066252231598, -0.026544300839304924, -0.027091361582279205, 0.057798076421022415, -0.0034756148234009743, -0.0028126018587499857, 0.03184840455651283, 0.010120609775185585, -0.0341080017387867, -0.04464485123753548, -0.03448856249451637, -0.009365429170429707, -0.02232242561876774, -0.0004307354101911187, -0.013248365372419357, 0.021335339173674583, -0.028542259708046913, 0.014699263498187065, 0.007979939691722393, 0.004073218442499638, 0.025735603645443916, -0.019123313948512077, 0.001523740473203361, 0.02275056019425392, 0.050567369908094406, -0.00046789980842731893, 0.006879874039441347, 0.046690378338098526, -0.010881736874580383, -0.023761430755257607, 0.04267067834734917, -0.03969752788543701, 0.025378825142979622, 0.03536861762404442, -0.007064209319651127, 0.0404110811650753, -0.057084519416093826, 0.020514748990535736, -0.004209983162581921, -0.06336382031440735, -0.002717460971325636, -0.04747529327869415, 0.010519011877477169, 0.01953955553472042, 0.00616037100

This is a simple example, sending a simple text to the embedding model and generating an embedding model in return. Notice this is a highly dimensional vector (1536 dims) in semantic space and we rely on semantic similarity through vector search looking for relevant context in RAG patterns. 

### CONCLUSION

With this we have completed the AI Foundry onboarding. 
- Created an AI Foundry Project 
- Deployed required LLM's into the project 
- Tested endpoints for gpt-4.1, o4-mini and embedv3 with simple API calls 


Now, although we typed in API keys and endpoint URL's directly into the cells, in production we need to use a secret vault service such as Azure Vault to store and fetch service / API keys. For simpliciy we will use a .env file in the repo. Now create a simple .env file and put your endpoint and API-key. We will adding further service keys to the .env file as we go along. 

```bash
# Azure OpenAI Configuration
AZURE_OPENAI_API_KEY="XXXX"
AZURE_OPENAI_ENDPOINT="XXXX"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4.1"
AZURE_OPENAI_REASONING_DEPLOYMENT_NAME="o4-mini"
AZURE_REASONING_OPENAI_API_VERSION="2025-04-01-preview"
AZURE_TEXT_EMBEDDING_DEPLOYMENT_NAME="text-embedding-3-small"
```

Notice the API versions for the completions API used in gpt-4.1 model and the responses API used with the reasoning models such as o4-mini are different. 
Also as previously stated, make sure the deployment names are kept generic not to keep the operation simple. 

Now let's move to 02_onboarding_ai_services.ipynb notebook, to start creating the AI Search service - a purpose built vector db we use to store embeddings for implementing RAG.
